# Content Edit - Version Tracking Test Data Insertion

## 🎯 Purpose

This notebook inserts **version tracking test data** into an existing `faq_update.db` database to simulate real-world content change scenarios.

### ✨ Test Scenarios Covered

1. **SCENARIO 1: Content Modification (Day 2)**
   - Page 2 content updated (sick leave policy changes)
   - Same page number, version incremented
   - Checksum changes to reflect new content

2. **SCENARIO 2: Page Insertion (Day 3)**
   - New page inserted at position 3 (remote work policy)
   - Subsequent pages shift down (+1)
   - Version numbers updated for shifted pages

3. **SCENARIO 3: Page Deletion (Optional - Commented Out)**
   - Page marked as Inactive
   - Subsequent pages shift up (-1)
   - Can be enabled for delete scenario testing

### 📋 What This Notebook Does

- ✅ Connects to existing `faq_update.db`
- ✅ Creates markdown files with test content
- ✅ Inserts new records simulating content changes
- ✅ Maintains proper checksums and timestamps
- ✅ Tracks operations for analysis

### ⚠️ Prerequisites

- Database must exist: `../databases/faq_update.db`
- Run `setup_tables_enhanced.ipynb` first if database doesn't exist

## 📖 When to Run

- Testing content change detection
- Validating version tracking logic
- Simulating real-world document updates

## Step 1: Configuration and Imports

In [1]:
import sqlite3
import os
import pandas as pd
import hashlib
from pathlib import Path
from datetime import datetime, timedelta
import sys

# ============================================================================
# 📁 PATH CONFIGURATION
# ============================================================================

# Notebook and project paths
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / 'src'

# Add src to Python path for utility imports
sys.path.insert(0, str(SRC_DIR))

# Directories
DATABASE_DIR = PROJECT_ROOT / 'databases'
TEST_DATA_DIR = PROJECT_ROOT / 'test_data_version_tracking'
MARKDOWN_DIR = TEST_DATA_DIR / 'markdown_files' / 'leave_policy'

# Database file
DATABASE_NAME = 'faq_update.db'
DB_PATH = DATABASE_DIR / DATABASE_NAME

# Create directories
MARKDOWN_DIR.mkdir(parents=True, exist_ok=True)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 60)

# ============================================================================
# 📊 DISPLAY CONFIGURATION
# ============================================================================

print("=" * 80)
print("🚀 CONTENT EDIT - Version Tracking Test Data Insertion")
print("=" * 80)
print(f"📁 Database: {DB_PATH.absolute()}")
print(f"📁 Test Data Directory: {TEST_DATA_DIR.absolute()}")
print(f"📁 Markdown Directory: {MARKDOWN_DIR.absolute()}")
print("=" * 80)

🚀 CONTENT EDIT - Version Tracking Test Data Insertion
📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\faq_update.db
📁 Test Data Directory: c:\Users\praka\projects\FAQ\FAQ_update\test_data_version_tracking
📁 Markdown Directory: c:\Users\praka\projects\FAQ\FAQ_update\test_data_version_tracking\markdown_files\leave_policy


## Step 2: Utility Functions

In [2]:
# Import utility functions from src/utils
from utils import (
    BASE_TIME,
    format_timestamp,
    get_timestamp,
    calculate_checksum,
    calculate_file_checksum,
    validate_checksum,
    create_markdown_file,
    compress_diff,
    decompress_diff,
    preview_dataframe,
    print_config_banner,
    log_success,
    log_info,
    log_warning,
    log_error
)

## Step 3: Define Test Content

In [3]:
# ============================================================================
# 📝 SAMPLE MARKDOWN CONTENT FOR TEST SCENARIOS
# ============================================================================

POLICY_CONTENT = {
    # SCENARIO 1: Modified content (page 2 updated on Day 2)
    'page2_v2': """## 4. Sick Leave (UPDATED)
Employees are entitled to sick leave as follows:
- **15 days** of paid sick leave per year (INCREASED FROM 10)
- Medical certificate required for absences over **2 consecutive days** (CHANGED FROM 3)
- Unused sick leave **can carry over up to 3 days** (NEW POLICY)

## 5. Parental Leave
- 12 weeks of paid parental leave for primary caregiver
- 4 weeks of paid parental leave for secondary caregiver
- **NEW: Additional 2 weeks unpaid leave available upon request**
""",

    # SCENARIO 2: New page inserted (becomes page 3 on Day 3, others shift)
    'page3_new': """## 5B. Remote Work During Parental Leave (NEW SECTION)
- Employees may request flexible remote work arrangements
- Available during final 4 weeks of parental leave
- Must be approved by manager and HR
- Subject to business needs and role requirements

## 5C. Parental Leave Return-to-Work Support
- Gradual return schedule available (min 20 hours/week)
- Lactation room access and break times provided
- Career development catch-up sessions offered
""",

    # Original page 3 content (will become page 4 on Day 3)
    'page3_v1': """## 6. Bereavement Leave
- 5 days for immediate family member
- 2 days for extended family member
- No documentation required

## 7. Leave Request Process
1. Submit request via HR portal
2. Manager approval required
3. Confirmation sent within 48 hours
""",
}

# Timeline base
BASE_TIME = datetime(2025, 1, 1, 10, 0, 0)

log_success("Test content definitions loaded")
log_info(f"Base timestamp: {BASE_TIME.strftime('%Y-%m-%d %H:%M:%S')}")

✅ Test content definitions loaded
ℹ️  Base timestamp: 2025-01-01 10:00:00


## Step 4: Verify Database Connection

In [4]:
print("\n" + "="*80)
print("DATABASE CONNECTION VERIFICATION")
print("="*80)

if not DB_PATH.exists():
    log_error(f"Database not found: {DB_PATH}")
    log_error("Please run 'setup_tables_enhanced.ipynb' first to create the database")
    raise FileNotFoundError(f"Database file not found: {DB_PATH}")

log_success(f"Database found: {DB_PATH.name}")

# Test connection and verify table
with sqlite3.connect(DB_PATH) as conn:
    # Check if content_repo table exists
    table_check = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='table' AND name='content_repo'
    """).fetchone()
    
    if not table_check:
        log_error("content_repo table not found in database")
        raise ValueError("Database missing content_repo table")
    
    log_success("content_repo table exists")
    
    # Check current record count
    count = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
    log_info(f"Current records in content_repo: {count}")
    
    # Show existing records if any
    if count > 0:
        existing_df = pd.read_sql_query("""
            SELECT ud_source_file_id, raw_file_nme, raw_file_page_nbr, 
                   version_nbr, file_status, last_modified_dt
            FROM content_repo
            ORDER BY ud_source_file_id
        """, conn)
        preview_dataframe(existing_df, "Existing content_repo records", max_rows=10)
    
    # Enable foreign keys
    conn.execute("PRAGMA foreign_keys = ON")
    fk_status = conn.execute("PRAGMA foreign_keys").fetchone()[0]
    if fk_status:
        log_success("Foreign keys ENABLED")
    else:
        log_warning("Foreign keys NOT enabled")


DATABASE CONNECTION VERIFICATION
✅ Database found: faq_update.db
✅ content_repo table exists
ℹ️  Current records in content_repo: 3

📄 Existing content_repo records Preview (first 3 of 3 rows):
--------------------------------------------------------------------------------
 ud_source_file_id              raw_file_nme  raw_file_page_nbr  version_nbr file_status         last_modified_dt
              1001 Employee_Leave_Policy.pdf                  1            1      Active 2025-10-20T03:46:37.000Z
              1002 Employee_Leave_Policy.pdf                  2            1      Active 2025-10-20T03:46:37.000Z
              1003 Employee_Leave_Policy.pdf                  3            1      Active 2025-10-20T03:46:37.000Z
--------------------------------------------------------------------------------
✅ Foreign keys ENABLED


## Step 5: SCENARIO 1 - Content Modification (Day 2)

In [5]:
print("\n" + "="*80)
print("📅 SCENARIO 1: CONTENT MODIFICATION (Day 2 - 2025-01-02)")
print("="*80)

log_info("Simulating page 2 content update (sick leave policy changes)")

# Create updated markdown file for page 2
md_path_p2_v2 = MARKDOWN_DIR / 'page2_v2.md'
checksum_p2_v2 = create_markdown_file(md_path_p2_v2, POLICY_CONTENT['page2_v2'])

log_success(f"Created markdown file: {md_path_p2_v2.name}")
log_info(f"Checksum: {checksum_p2_v2}")

now = format_timestamp(use_base_time=False)

# Prepare record for page 2 version 2
scenario1_record = {
    'ud_source_file_id': 1004,
    'domain': 'HR',
    'service': 'Policy Management',
    'orgoid': 'ORG001',
    'associateoid': 'ASSOC001',
    'raw_file_nme': 'Employee_Leave_Policy.pdf',
    'raw_file_type': '.pdf',
    'raw_file_version_nbr': 2,  # Version incremented
    'source_url_txt': 'https://intranet.example.com/policies/leave',
    'parent_location_txt': '/policies',
    'raw_file_path': '/test/Employee_Leave_Policy.pdf',
    'extracted_markdown_file_path': str(md_path_p2_v2.absolute()),
    'extracted_layout_file_path': 'layout_p2_v2.txt',
    'raw_file_page_nbr': 2,  # SAME page number
    'title_nme': 'Employee Leave Policy - Sick Leave',  # More specific
    'breadcrumb_txt': 'Policies > HR > Leave Management',  # Added breadcrumbs
    'content_tags_txt': 'leave;sick;medical;absence',  # Added tags
    'version_nbr': 2,  # Version incremented
    'content_checksum': checksum_p2_v2,  # NEW checksum
    'file_status': 'Active',
    'created_dt': get_timestamp(0, base_time=BASE_TIME),  # Original creation date (Day 0)
    'last_modified_dt': now,  # Day 1 (simulated)
}

df_scenario1 = pd.DataFrame([scenario1_record])

log_success("Scenario 1 record prepared")
preview_dataframe(
    df_scenario1[['ud_source_file_id', 'raw_file_page_nbr', 'version_nbr', 
                  'content_checksum']], 
    "Scenario 1: Content Modification"
)

print("\n💡 Key Changes:")
print("   - Page number: 2 (unchanged)")
print("   - Version: 1 → 2")
print("   - Checksum: Modified (reflects content update)")
print("   - Sick leave days: 10 → 15 days")
print("   - Medical cert required: 3 → 2 consecutive days")
print("   - Carryover: New policy (up to 3 days)")


📅 SCENARIO 1: CONTENT MODIFICATION (Day 2 - 2025-01-02)
ℹ️  Simulating page 2 content update (sick leave policy changes)
✅ Created markdown file: page2_v2.md
ℹ️  Checksum: ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef
✅ Scenario 1 record prepared

📄 Scenario 1: Content Modification Preview (first 1 of 1 rows):
--------------------------------------------------------------------------------
 ud_source_file_id  raw_file_page_nbr  version_nbr                                                 content_checksum
              1004                  2            2 ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef
--------------------------------------------------------------------------------

💡 Key Changes:
   - Page number: 2 (unchanged)
   - Version: 1 → 2
   - Checksum: Modified (reflects content update)
   - Sick leave days: 10 → 15 days
   - Medical cert required: 3 → 2 consecutive days
   - Carryover: New policy (up to 3 days)


## Step 6: SCENARIO 2 - Page Insertion (Day 3)

In [6]:
print("\n" + "="*80)
print("📅 SCENARIO 2: PAGE INSERTION (Day 3 - 2025-01-03)")
print("="*80)

log_info("Simulating new page insertion at position 3")

# Create new page 3 markdown
md_path_p3_v2 = MARKDOWN_DIR / 'page3_v2.md'
checksum_p3_v2 = create_markdown_file(md_path_p3_v2, POLICY_CONTENT['page3_new'])

log_success(f"Created markdown file: {md_path_p3_v2.name}")
log_info(f"Checksum: {checksum_p3_v2}")

# Create old page 3 markdown (will shift to page 4)
md_path_p4_v1 = MARKDOWN_DIR / 'page3_v1.md'
checksum_p4_v1 = create_markdown_file(md_path_p4_v1, POLICY_CONTENT['page3_v1'])

log_success(f"Created markdown file: {md_path_p4_v1.name}")
log_info(f"Checksum: {checksum_p4_v1}")

# Record 1: New page 3 (inserted)
scenario2_record1 = {
    'ud_source_file_id': 1005,  # NEW ID
    'domain': 'HR',
    'service': 'Policy Management',
    'orgoid': 'ORG001',
    'associateoid': 'ASSOC001',
    'raw_file_nme': 'Employee_Leave_Policy.pdf',
    'raw_file_type': '.pdf',
    'raw_file_version_nbr': 2,
    'source_url_txt': 'https://intranet.example.com/policies/leave',
    'parent_location_txt': '/policies',
    'raw_file_path': '/test/Employee_Leave_Policy.pdf',
    'extracted_markdown_file_path': str(md_path_p3_v2.absolute()),
    'extracted_layout_file_path': 'layout_p3_new.txt',
    'raw_file_page_nbr': 3,  # Inserted at position 3
    'title_nme': 'Employee Leave Policy - Remote Work During Parental Leave',  # Specific
    'breadcrumb_txt': 'Policies > HR > Leave Management',  # Added breadcrumbs
    'content_tags_txt': 'leave;parental;remote;flexible',  # Added tags
    'version_nbr': 2,
    'content_checksum': checksum_p3_v2,
    'file_status': 'Active',
    'created_dt': get_timestamp(2, base_time=BASE_TIME),  # Day 2
    'last_modified_dt': now,  # Day 2 (consistent)
}

# Record 2: Old page 3 → page 4 (shifted)
scenario2_record2 = {
    'ud_source_file_id': 1006,
    'domain': 'HR',
    'service': 'Policy Management',
    'orgoid': 'ORG001',
    'associateoid': 'ASSOC001',
    'raw_file_nme': 'Employee_Leave_Policy.pdf',
    'raw_file_type': '.pdf',
    'raw_file_version_nbr': 1,
    'source_url_txt': 'https://intranet.example.com/policies/leave',
    'parent_location_txt': '/policies',
    'raw_file_path': '/test/Employee_Leave_Policy.pdf',
    'extracted_markdown_file_path': str(md_path_p4_v1.absolute()),  # SAME file
    'extracted_layout_file_path': 'layout_p3.txt',
    'raw_file_page_nbr': 4,  # Shifted from 3 to 4
    'title_nme': 'Employee Leave Policy - Bereavement and Leave Request Process',  # Specific
    'breadcrumb_txt': 'Policies > HR > Leave Management',  # Added breadcrumbs
    'content_tags_txt': 'leave;bereavement;process;request',  # Added tags
    'version_nbr': 1,
    'content_checksum': checksum_p4_v1,  # SAME checksum (content unchanged)
    'file_status': 'Active',
    'created_dt': get_timestamp(0, base_time=BASE_TIME),  # Original creation (Day 0)
    'last_modified_dt': now,  # Day 2 (position updated)
}

df_scenario2 = pd.DataFrame([scenario2_record1, scenario2_record2])

log_success("Scenario 2 records prepared")
preview_dataframe(
    df_scenario2[['ud_source_file_id', 'raw_file_page_nbr', 'version_nbr', 
                  'content_checksum']], 
    "Scenario 2: Page Insertion"
)

print("\n💡 Key Changes:")
print("   - New page 3: Remote Work During Parental Leave (inserted)")
print("   - Old page 3 → page 4: Bereavement Leave (shifted)")
print("   - Version tracking: Insertion requires page renumbering")


📅 SCENARIO 2: PAGE INSERTION (Day 3 - 2025-01-03)
ℹ️  Simulating new page insertion at position 3
✅ Created markdown file: page3_v2.md
ℹ️  Checksum: 4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca667c067d
✅ Created markdown file: page3_v1.md
ℹ️  Checksum: 1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca6b9abe19
✅ Scenario 2 records prepared

📄 Scenario 2: Page Insertion Preview (first 2 of 2 rows):
--------------------------------------------------------------------------------
 ud_source_file_id  raw_file_page_nbr  version_nbr                                                 content_checksum
              1005                  3            2 4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca667c067d
              1006                  4            1 1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca6b9abe19
--------------------------------------------------------------------------------

💡 Key Changes:
   - New page 3: Remote Work During Parental Leave (in

## Step 7: SCENARIO 3 - Page Deletion (Optional - Commented Out)

In [7]:
print("\n" + "="*80)
print("📅 SCENARIO 3: PAGE DELETION (OPTIONAL - Currently Disabled)")
print("="*80)

log_warning("Scenario 3 (DELETE) is commented out")
log_info("This scenario can be enabled for delete testing by uncommenting the code below")

# Uncomment the following section to enable Scenario 3

# # ============================================================================
# # SCENARIO 3: DELETE PAGE (Day 4)
# # ============================================================================
# 
# log_info("Simulating page 2 deletion (subsequent pages shift up)")
# 
# # Record 1: Mark page 2 as Inactive (deleted)
# scenario3_record1 = {
#     'ud_source_file_id': 1002,  # SAME ID from original page 2
#     'domain': 'HR',
#     'service': 'Policy Management',
#     'orgoid': 'ORG001',
#     'associateoid': 'ASSOC001',
#     'raw_file_nme': 'Employee_Leave_Policy.pdf',
#     'raw_file_type': '.pdf',
#     'raw_file_version_nbr': 2,
#     'source_url_txt': None,
#     'parent_location_txt': None,
#     'raw_file_path': '/test/Employee_Leave_Policy.pdf',
#     'extracted_markdown_file_path': str(md_path_p2_v2.absolute()),
#     'extracted_layout_file_path': 'layout_p2_v2.txt',
#     'raw_file_page_nbr': 2,
#     'title_nme': 'Employee Leave Policy',
#     'breadcrumb_txt': None,
#     'content_tags_txt': None,
#     'version_nbr': 2,
#     'content_checksum': checksum_p2_v2,
#     'file_status': 'Inactive',  # DELETED
#     'created_dt': get_timestamp(0, base_time=BASE_TIME),
#     'last_modified_dt': now,  # Day 4
#     'operation': 'DELETE',
#     'operation_date': '2025-01-04'
# }
# 
# # Additional records for page shifting would go here...
# # (Page 3 → Page 2, Page 4 → Page 3, etc.)
# 
# df_scenario3 = pd.DataFrame([scenario3_record1])
# 
# log_success("Scenario 3 records prepared")
# preview_dataframe(
#     df_scenario3[['ud_source_file_id', 'raw_file_page_nbr', 'version_nbr', 
#                   'file_status', 'operation', 'operation_date']], 
#     "Scenario 3: Page Deletion"
# )

# For now, create empty DataFrame
df_scenario3 = pd.DataFrame()

print("\n💡 To enable Scenario 3:")
print("   1. Uncomment the code in this cell")
print("   2. Re-run this cell and subsequent cells")
print("   3. Test page deletion and shift-up behavior")


📅 SCENARIO 3: PAGE DELETION (OPTIONAL - Currently Disabled)
⚠️  Scenario 3 (DELETE) is commented out
ℹ️  This scenario can be enabled for delete testing by uncommenting the code below

💡 To enable Scenario 3:
   1. Uncomment the code in this cell
   2. Re-run this cell and subsequent cells
   3. Test page deletion and shift-up behavior


## Step 8: Combine All Test Records

In [8]:
print("\n" + "="*80)
print("COMBINING TEST RECORDS")
print("="*80)

# Combine all scenarios
all_scenarios = []

if not df_scenario1.empty:
    all_scenarios.append(df_scenario1)
    log_info(f"Scenario 1 (UPDATE): {len(df_scenario1)} record(s)")

if not df_scenario2.empty:
    all_scenarios.append(df_scenario2)
    log_info(f"Scenario 2 (INSERT): {len(df_scenario2)} record(s)")

if not df_scenario3.empty:
    all_scenarios.append(df_scenario3)
    log_info(f"Scenario 3 (DELETE): {len(df_scenario3)} record(s)")

if not all_scenarios:
    log_error("No test scenarios to insert!")
    raise ValueError("No data to insert")

# Combine into single DataFrame
df_all_records = pd.concat(all_scenarios, ignore_index=True)

log_success(f"Combined {len(df_all_records)} total test records")

# Show summary with operation tracking
preview_dataframe(
    df_all_records[['ud_source_file_id', 'raw_file_page_nbr', 'version_nbr', 
                    'file_status', 'last_modified_dt']], 
    "All Test Records Summary"
)


COMBINING TEST RECORDS
ℹ️  Scenario 1 (UPDATE): 1 record(s)
ℹ️  Scenario 2 (INSERT): 2 record(s)
✅ Combined 3 total test records

📄 All Test Records Summary Preview (first 3 of 3 rows):
--------------------------------------------------------------------------------
 ud_source_file_id  raw_file_page_nbr  version_nbr file_status         last_modified_dt
              1004                  2            2      Active 2025-10-20T03:47:22.000Z
              1005                  3            2      Active 2025-10-20T03:47:22.000Z
              1006                  4            1      Active 2025-10-20T03:47:22.000Z
--------------------------------------------------------------------------------


## Step 9: Insert Records into Database

In [9]:
print("\n" + "="*80)
print("DATABASE INSERTION")
print("="*80)

with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Get count before insertion
    count_before = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
    log_info(f"Records before insertion: {count_before}")
    
    # Prepare data for insertion (drop operation tracking columns)
    # df_to_insert = df_all_records.drop(columns=['operation', 'operation_date'])
    df_to_insert = df_all_records
    
    try:
        # Insert main records
        df_to_insert.to_sql('content_repo', conn, if_exists='append', index=False)
        log_success(f"Inserted {len(df_to_insert)} records into content_repo")
        
        # Also insert into tracking table (with operation info)
        df_all_records.to_sql(
            'content_repo_test_operations', 
            conn, 
            if_exists='replace',  # Replace each run for clean tracking
            index=False
        )
        log_success("Created content_repo_test_operations table for operation tracking")
        
        conn.commit()
        
        # Get count after insertion
        count_after = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
        log_success(f"Records after insertion: {count_after}")
        log_info(f"New records added: {count_after - count_before}")
        
    except sqlite3.IntegrityError as e:
        log_error(f"Database integrity error: {e}")
        log_warning("Possible causes:")
        log_warning("  - Duplicate ud_source_file_id")
        log_warning("  - Duplicate (raw_file_nme, raw_file_page_nbr) combination")
        log_warning("  - Foreign key constraint violation")
        raise
    except Exception as e:
        log_error(f"Unexpected error during insertion: {e}")
        raise


DATABASE INSERTION
ℹ️  Records before insertion: 3
✅ Inserted 3 records into content_repo
✅ Created content_repo_test_operations table for operation tracking
✅ Records after insertion: 6
ℹ️  New records added: 3


## Step 9.5: Foreign Key Validation

In [10]:
print("\n" + "="*80)
print("FOREIGN KEY VALIDATION")
print("="*80)

with sqlite3.connect(DB_PATH) as conn:
    # Check foreign key integrity
    fk_check = pd.read_sql("PRAGMA foreign_key_check", conn)
    
    if fk_check.empty:
        log_success("✅ All foreign key constraints valid")
        log_info("All inserted records have valid foreign key relationships")
    else:
        log_error(f"❌ Foreign key violations found: {len(fk_check)}")
        print("\n⚠️  Foreign Key Violations Detected:")
        preview_dataframe(fk_check, "FK Violations")
        
        # Provide detailed error information
        for idx, row in fk_check.iterrows():
            print(f"\n   Violation {idx+1}:")
            print(f"   - Table: {row.get('table', 'N/A')}")
            print(f"   - Row ID: {row.get('rowid', 'N/A')}")
            print(f"   - Parent Table: {row.get('parent', 'N/A')}")
            print(f"   - FK Index: {row.get('fkid', 'N/A')}")
        
        log_warning("Database integrity compromised - please review insertions")
    
    # Additional checks: Verify content_repo records don't violate uniqueness
    duplicate_check = conn.execute("""
        SELECT raw_file_nme, raw_file_page_nbr, version_nbr, COUNT(*) as count
        FROM content_repo
        GROUP BY raw_file_nme, raw_file_page_nbr, version_nbr
        HAVING COUNT(*) > 1
    """).fetchall()
    
    if duplicate_check:
        log_warning(f"⚠️  Found {len(duplicate_check)} potential duplicate(s)")
        for row in duplicate_check:
            print(f"   - File: {row[0]}, Page: {row[1]}, Version: {row[2]}, Count: {row[3]}")
    else:
        log_success("✅ No duplicate (file, page, version) combinations found")

print("\n💡 Foreign Key Validation Complete")
print("   This ensures data integrity across related tables")


FOREIGN KEY VALIDATION
✅ ✅ All foreign key constraints valid
ℹ️  All inserted records have valid foreign key relationships
✅ ✅ No duplicate (file, page, version) combinations found

💡 Foreign Key Validation Complete
   This ensures data integrity across related tables


## Step 10: Verify Inserted Data

In [11]:
print("\n" + "="*80)
print("DATA VERIFICATION")
print("="*80)

with sqlite3.connect(DB_PATH) as conn:
    # Query all records from content_repo
    all_records_df = pd.read_sql_query("""
        SELECT *
        FROM content_repo
        ORDER BY ud_source_file_id
    """, conn)
    
    log_success(f"Total records in content_repo: {len(all_records_df)}")
    preview_dataframe(all_records_df, "All Records in content_repo", max_rows=15)
    
    # Show operation tracking table
    tracking_check = conn.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='table' AND name='content_repo_test_operations'
    """).fetchone()
    
    if tracking_check:
        tracking_df = pd.read_sql_query("""
            SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
                   file_status
            FROM content_repo_test_operations
            ORDER BY ud_source_file_id
        """, conn)
        
        preview_dataframe(tracking_df, "Operation Tracking Records", max_rows=15)
    
    # Check for specific scenarios
    print("\n📋 Scenario Verification:")
    
    # Scenario 1: Page 2 update
    page2_records = pd.read_sql_query("""
        SELECT ud_source_file_id, raw_file_page_nbr, version_nbr, 
               content_checksum, last_modified_dt
        FROM content_repo
        WHERE raw_file_page_nbr = 2
        ORDER BY ud_source_file_id
    """, conn)
    
    if len(page2_records) > 0:
        log_success(f"✓ Scenario 1: Found {len(page2_records)} page 2 record(s)")
        for _, row in page2_records.iterrows():
            print(f"    - ID {row['ud_source_file_id']}: v{row['version_nbr']}, checksum: {row['content_checksum'][:16]}...")
    
    # Scenario 2: Page 3 and 4
    page3_records = pd.read_sql_query("""
        SELECT ud_source_file_id, raw_file_page_nbr, version_nbr
        FROM content_repo
        WHERE raw_file_page_nbr IN (3, 4)
        ORDER BY raw_file_page_nbr
    """, conn)
    
    if len(page3_records) > 0:
        log_success(f"✓ Scenario 2: Found {len(page3_records)} page 3/4 record(s)")
        for _, row in page3_records.iterrows():
            print(f"    - ID {row['ud_source_file_id']}: Page {row['raw_file_page_nbr']}, v{row['version_nbr']}")
    
    # Check for inactive records (Scenario 3)
    inactive_count = conn.execute("""
        SELECT COUNT(*) FROM content_repo WHERE file_status = 'Inactive'
    """).fetchone()[0]
    
    if inactive_count > 0:
        log_success(f"✓ Scenario 3: Found {inactive_count} inactive record(s)")
    else:
        log_info("Scenario 3: No inactive records (DELETE scenario not enabled)")


DATA VERIFICATION
✅ Total records in content_repo: 6

📄 All Records in content_repo Preview (first 6 of 6 rows):
--------------------------------------------------------------------------------
 ud_source_file_id domain           service orgoid associateoid              raw_file_nme raw_file_type  raw_file_version_nbr  raw_file_page_nbr                              source_url_txt parent_location_txt                   raw_file_path                                                                                      extracted_markdown_file_path extracted_layout_file_path                                                     title_nme                   breadcrumb_txt                  content_tags_txt  version_nbr                                                 content_checksum file_status               created_dt         last_modified_dt
              1001     HR Policy Management ORG001     ASSOC001 Employee_Leave_Policy.pdf          .pdf                     1                  1          

In [12]:
all_records_df[['ud_source_file_id', 'raw_file_page_nbr', 'version_nbr', 'file_status', 'content_checksum', 'last_modified_dt']]

ud_source_file_id  raw_file_page_nbr  version_nbr file_status  \
0               1001                  1            1      Active   
1               1002                  2            1      Active   
2               1003                  3            1      Active   
3               1004                  2            2      Active   
4               1005                  3            2      Active   
5               1006                  4            1      Active   

                                              content_checksum  \
0  620c75e45bb1c3a1225a411c9821805c866c60fe8e6a61c8586ab3ac...   
1  b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba37639...   
2  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   
3  ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1...   
4  4aefd3fb027461ca50f27186787cc28e0d1a90c91ed4764561f509ca...   
5  1e4bd8f57457cee3cb79d7b43f8e8cd4853e7360146738a1de139fca...   

           last_modified_dt  
0  2025-10-20T03:46:37.000Z  
1  2025-10-20T03:46:37.000Z  
2  2025-10-20T03:46:37.000Z  
3  2025-10-20T03:47:22.000Z  
4  2025-10-20T03:47:22.000Z  
5  2025-10-20T03:47:22.000Z

## Step 11: Summary and Next Steps

In [13]:
print("\n" + "="*80)
print("✅ CONTENT EDIT COMPLETE")
print("="*80)

with sqlite3.connect(DB_PATH) as conn:
    total_records = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
    active_records = conn.execute(
        "SELECT COUNT(*) FROM content_repo WHERE file_status = 'Active'"
    ).fetchone()[0]
    inactive_records = conn.execute(
        "SELECT COUNT(*) FROM content_repo WHERE file_status = 'Inactive'"
    ).fetchone()[0]
    
    print(f"\n📁 Database: {DB_PATH.absolute()}")
    print(f"\n📊 Database Statistics:")
    print(f"   Total records:    {total_records}")
    print(f"   Active records:   {active_records}")
    print(f"   Inactive records: {inactive_records}")
    
    # File size
    if DB_PATH.exists():
        db_size = os.path.getsize(DB_PATH)
        print(f"\n💾 Database Size: {db_size:,} bytes ({db_size/1024:.2f} KB)")
    
    # Markdown files created
    md_files = list(MARKDOWN_DIR.glob('*.md'))
    print(f"\n📝 Markdown Files Created: {len(md_files)}")
    for md_file in sorted(md_files):
        file_size = os.path.getsize(md_file)
        print(f"   - {md_file.name} ({file_size} bytes)")

print("\n📖 Test Scenarios Executed:")
print("   ✅ Scenario 1: Content Modification (Page 2 update)")
print("   ✅ Scenario 2: Page Insertion (New page 3, shift page 3→4)")
print("   ⚠️  Scenario 3: Page Deletion (Commented out - can be enabled)")

print("\n📖 Next Steps:")
print("   1. Run change detection logic to identify modifications")
print("   2. Test version tracking and diff generation")
print("   3. Validate FAQ invalidation based on content changes")
print("   4. Analyze operation tracking table for audit trail")

print("\n💡 Tips:")
print("   - Use content_repo_test_operations table to see operation timeline")
print("   - Compare checksums to detect content modifications")
print("   - Enable Scenario 3 in Step 7 to test page deletion")

print("\n" + "="*80)
print("🎉 TEST DATA INSERTION SUCCESSFUL!")
print("="*80)


✅ CONTENT EDIT COMPLETE

📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\faq_update.db

📊 Database Statistics:
   Total records:    6
   Active records:   6
   Inactive records: 0

💾 Database Size: 245,760 bytes (240.00 KB)

📝 Markdown Files Created: 3
   - page2_v2.md (493 bytes)
   - page3_v1.md (252 bytes)
   - page3_v2.md (449 bytes)

📖 Test Scenarios Executed:
   ✅ Scenario 1: Content Modification (Page 2 update)
   ✅ Scenario 2: Page Insertion (New page 3, shift page 3→4)
   ⚠️  Scenario 3: Page Deletion (Commented out - can be enabled)

📖 Next Steps:
   1. Run change detection logic to identify modifications
   2. Test version tracking and diff generation
   3. Validate FAQ invalidation based on content changes
   4. Analyze operation tracking table for audit trail

💡 Tips:
   - Use content_repo_test_operations table to see operation timeline
   - Compare checksums to detect content modifications
   - Enable Scenario 3 in Step 7 to test page deletion

🎉 TEST DATA I